<span style="font-size:3.0em">
    Wikipedia Scrape – Tony Nominees
    <br>
</span> 

<i> Some neat smooth functions for grabbing data from Wikipedia. There are some inadequacies, skip down to section 6 to read what they are.</i>

# Import Functions and Stuff

## Dependancies

In [1]:
import requests
import pandas as pd
from IPython.display import display_html
from bs4 import BeautifulSoup
import re
import os

## Function to get tables from url

<pre>
get_tables_from_url(url)
</pre>

In [2]:

def get_tables_from_url(url):
    """
    Gets a list of tables from an input url.
    
    """

    # Request the document
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc,'lxml')
    
    table_classes = {"class": ["wikitable"]}
    tables = soup.findAll("table", table_classes)
    
    return tables

## Function to get df from tables
<pre>
get_df_from_table(table_in)
</pre>

In [3]:
def get_df_from_table(table_in):
    """
    returns a structured table for a beautiful soup table
    """
    
    ### SET UP REGEX
    
    re_1 = "^\d{4}"
    re_2 = "\d{1,2}(th|rd|st|nd) Tony Awards"


    # JUMP INTO IT!
    parsed_table_data=[]
    rows = table_in.findAll('tr')
    
    row_1 = rows[0]
    children = row_1.findChildren(recursive=False)
    r_n = len(children)

    parsed_table_data = []
    for row in rows:
        children = row.findChildren(recursive=False)

        row_text = []
        for child in children:

            clean_text = child.text

            # We can sort each row as follows:
            if re.findall(re_2,clean_text):
                string  = clean_text
                
                # get the year
                yr = re.match(re_1,string).group()
                # split the string
                string_2 = string.split(yr)[1]
                # get the tony year
                t_yr = re.match(re_2,string_2).group()
                
   

                # Send the cleaned text back to the loop
                clean_text = yr +' | ' + t_yr
                row_text.append(clean_text)
                
                continue

            #This is to discard reference/citation links
            clean_text = clean_text.split('&#91;')[0]
            #This is to clean the header row of the sort icons
            clean_text = clean_text.split('&#160;')[-1]
            # This discards demarcation of winner of pulitzer prize
            clean_text = clean_text.replace('†','')

            clean_text = clean_text.strip()
            row_text.append(clean_text)
            

        if '|' in row_text[0]:
            cell_1 = row_text[0]
            continue
            
        else:
            if 'cell_1' not in dir():
                cell_1 = 'nothing'
            row_text.insert(0, cell_1)

        if len(row_text) >r_n:
            # removes first item, we don't want it...
            row_text.pop(0)

        if len(row_text) <r_n:
            x = row_text[-1]
            # How many items are you missing? (diff)
            diff = r_n - len(row_text)
            for i in range(diff):
                # for each item missing, append
                row_text.append(x)

        parsed_table_data.append(row_text)
    
    ## ONTO PART II
    headers = parsed_table_data.pop(0)
    df = pd.DataFrame(parsed_table_data, columns=headers)
    return df

# Get Musicals
Scrape that wikipedia stuff.

In [4]:
url = 'https://en.wikipedia.org/wiki/Tony_Award_for_Best_Musical'

In [5]:
tables = get_tables_from_url(url)
df=pd.DataFrame()
for table in tables:
    sub_df = get_df_from_table(table)
    df = df.append(sub_df,ignore_index=True)
    
name_root = url.split('/')[-1]
df_name = 'Wikipedia_scrape_' + name_root + '.csv'
df.to_csv(df_name)

# Get Plays
Scrape that wikipedia stuff.

In [6]:
url = 'https://en.wikipedia.org/wiki/Tony_Award_for_Best_Play'

In [7]:
tables = get_tables_from_url(url)
df=pd.DataFrame()
for table in tables:
    sub_df = get_df_from_table(table)
    df = df.append(sub_df,ignore_index=True)
    
name_root = url.split('/')[-1]
df_name = 'Wikipedia_scrape_' + name_root + '.csv'
df.to_csv(df_name)

# Get Revival of a Musical
Scrape that wikipedia stuff.

In [8]:
url = 'https://en.wikipedia.org/wiki/Tony_Award_for_Best_Revival_of_a_Musical'

In [9]:
tables = get_tables_from_url(url)
df=pd.DataFrame()
for table in tables:
    sub_df = get_df_from_table(table)
    df = df.append(sub_df,ignore_index=True)
    
name_root = url.split('/')[-1]
df_name = 'Wikipedia_scrape_' + name_root + '.csv'
df.to_csv(df_name)

# Get Revival of Plays
Scrape that wikipedia stuff.

In [10]:
url = 'https://en.wikipedia.org/wiki/Tony_Award_for_Best_Revival_of_a_Play'

In [11]:
tables = get_tables_from_url(url)
df=pd.DataFrame()
for table in tables:
    sub_df = get_df_from_table(table)
    df = df.append(sub_df,ignore_index=True)
    
name_root = url.split('/')[-1]
df_name = 'Wikipedia_scrape_' + name_root + '.csv'
df.to_csv(df_name)

# Finished!
Done! Each is in its own df.

<hr>
<i>
    Maybe make a single df with a column representing the category of a tony nominee? Also, would be nice to include which ones are winners. This can be done in the next iteration.  
    </i>
    <hr>

## Load df
<i>
If you want to load the files you just downloaded... 
</i>
<pre>
get_files(dir_in = os.getcwd(), csv_in = True):
</pre>

In [13]:
import os

def get_files(dir_in = os.getcwd(), csv_in = True):
    """
    returns a list of paths of files
    option to include csvs
    requires you load os
    """
    files = [x for x in os.listdir(dir_in)]
    file_paths = [os.path.abspath(x) for x in os.listdir(dir_in)]
    csv_paths = [x for x in file_paths if x.endswith('.csv')]
    
    if csv_in is False:
        return file_paths
    else:
        return csv_paths
        

In [16]:
import pandas as pd

csvs = get_files()
for x in csvs:
    if x.endswith('Wikipedia_scrape_Tony_Award_for_Best_Musical.csv'):
        df = pd.read_csv(x,index_col=0)
        print ('file is here!')
        print(df.head(5))
    else:
        print ('the file you want isn\'t here')

file is here!
                     Year         Musical  \
0  1949 | 3rd Tony Awards   Kiss Me, Kate   
1  1950 | 4th Tony Awards   South Pacific   
2  1951 | 5th Tony Awards  Guys and Dolls   
3  1952 | 6th Tony Awards  The King and I   
4  1953 | 7th Tony Awards  Wonderful Town   

                                  Book              Music  \
0               Bella & Samuel Spewack        Cole Porter   
1  Oscar Hammerstein II & Joshua Logan    Richard Rodgers   
2            Abe Burrows & Jo Swerling      Frank Loesser   
3                 Oscar Hammerstein II    Richard Rodgers   
4      Jerome Chodorov & Joseph Fields  Leonard Bernstein   

                        Lyrics  
0                  Cole Porter  
1                  Hammerstein  
2                Frank Loesser  
3                  Hammerstein  
4  Betty Comden & Adolph Green  
the file you want isn't here
the file you want isn't here
the file you want isn't here
